# Lectura 31: Streaming API

In [1]:
import polars as pl
from polars import col

import polars as pl
from polars import col

query = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .filter(col('MONTH') > 6)
    .group_by('AIRLINE').agg(
        media_tiempo_aire=col('AIR_TIME').mean()
    )
)

vuelos_agregado = query.collect(streaming=True)

vuelos_agregado

AIRLINE,media_tiempo_aire
str,f64
"""F9""",130.735121
"""NK""",132.105457
"""OO""",76.150833
"""WN""",102.733758
"""MQ""",64.836925
…,…
"""EV""",71.426605
"""DL""",115.037619
"""B6""",143.185472


In [2]:
print(query.explain(streaming=True))

--- STREAMING
AGGREGATE
	[col("AIR_TIME").mean().alias("media_tiempo_aire")] BY [col("AIRLINE")] FROM

    Parquet SCAN ./data/vuelos/vuelos.parquet
    PROJECT 3/31 COLUMNS
    SELECTION: [(col("MONTH")) > (6)]  --- END STREAMING

  DF []; PROJECT */0 COLUMNS; SELECTION: "None"


/var/folders/44/cd4ysnrs1zvgk8nff92w71lw0000gn/T/ipykernel_4828/3038726313.py:1: UserWarning: Cannot combine 'streaming' with 'comm_subplan_elim'. CSE will be turned off.
  print(query.explain(streaming=True))


In [3]:
query_2 = (
    pl.scan_parquet('./data/vuelos/vuelos.parquet')
    .with_columns(
        col('AIR_TIME').mean().over('AIR_LINE')
    )
)

print(query_2.explain(streaming=True))

 WITH_COLUMNS:
 [col("AIR_TIME").mean().over([col("AIR_LINE")])]
  --- STREAMING

  Parquet SCAN ./data/vuelos/vuelos.parquet
  PROJECT */31 COLUMNS  --- END STREAMING

    DF []; PROJECT */0 COLUMNS; SELECTION: "None"


/var/folders/44/cd4ysnrs1zvgk8nff92w71lw0000gn/T/ipykernel_4828/2954960651.py:8: UserWarning: Cannot combine 'streaming' with 'comm_subplan_elim'. CSE will be turned off.
  print(query_2.explain(streaming=True))
